In [1]:
import xarray
import fsspec
import numpy as np
import s3fs
import proplot
import os

from matplotlib import pyplot as plt
import pandas

/tube1/cha674/Anaconda_Install/miniconda/envs/py3/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = other[key]


In [2]:
def create_profile_index(wod_dataset,variable_name):
    
    cast_row_size = wod_dataset[variable_name + '_row_size'].values
    
    
    cast_counter = cast_row_size.cumsum().astype(int)
    
    cast_counter = np.concatenate([np.zeros([1],dtype=int),cast_counter])
    cast_index   = xarray.full_like(wod_dataset[variable_name + '_obs'],fill_value=0,dtype=int)
    
    #unique_cast_for_variable = 
    cast_row_counter = 0
    for i_cast in range(0,wod_dataset['casts'].size):

        current_cast_row_size = cast_row_size[i_cast] #.astype(int)
        #print(current_cast_row_size)
        if not np.isnan(current_cast_row_size):
            
            
            cast_index[cast_row_counter:cast_row_counter+int(current_cast_row_size)] =  wod_dataset['casts'][i_cast].astype(int)
            cast_row_counter = cast_row_counter + int(current_cast_row_size)
            
            
    return cast_index

In [3]:
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False, 
                           config_kwargs = {'max_pool_connections': 50})
    return s3fs.S3Map(path, s3=fs)

In [4]:
START_YEAR = 2018
END_YEAR   = 2018

#year = 2018
obs_type = ['ctd','xbt','pfl','mrb','gld','drb'] #,'gld']
base_s3_url = 's3://noaa-wod-pds'

data_container = []

for i_year in range(START_YEAR,END_YEAR+1):

    data_container_for_year = []
    for i_obs_type in obs_type:
        url = base_s3_url + '/' + str(i_year) + '/wod_' + i_obs_type + '_' + str(i_year) + '.nc' #_pfl_2018.nc'
        print(url)
        fs = s3fs.S3FileSystem(anon=True)
        file_obj = fs.open(url)
#ncfile = fsspec.open(url)
        ds = xarray.open_dataset(file_obj,engine='h5netcdf')
        data_container_for_year.append(ds)
    data_container.append(data_container_for_year)

#ds.air_temperature_at_2_metres.encoding


s3://noaa-wod-pds/2018/wod_ctd_2018.nc
s3://noaa-wod-pds/2018/wod_xbt_2018.nc
s3://noaa-wod-pds/2018/wod_pfl_2018.nc
s3://noaa-wod-pds/2018/wod_mrb_2018.nc
s3://noaa-wod-pds/2018/wod_gld_2018.nc
s3://noaa-wod-pds/2018/wod_drb_2018.nc


In [6]:
variable_name = 'Temperature'

for i_year in range(START_YEAR,END_YEAR+1):
    year_index = i_year-START_YEAR
    print(i_year)
    for i_obs_type in range(0,len(obs_type)):

        print(obs_type[i_obs_type])
        cast_temperature_index = create_profile_index(data_container[year_index][i_obs_type],variable_name)
        cast_depth_index       = create_profile_index(data_container[year_index][i_obs_type],'z')
        #dsadsa
        data_container[year_index][i_obs_type][variable_name + '_cast_index'] = cast_temperature_index
        data_container[year_index][i_obs_type]['z_cast_index']                = cast_depth_index

2018
ctd
xbt
pfl
mrb
gld
drb


In [11]:
LONGITUDE_LIM_EAST = 150
LONGITUDE_LIM_WEST = 140

LATITUDE_LIM_SOUTH = -50
LATITUDE_LIM_NORTH = -40
#obs_type = ['ctd','xbt','pfl','osd','mrb','gld','drb'] #,'gld']


year_to_get = 2018
i_year_to_get = year_to_get-START_YEAR

data_container_for_year = data_container[i_year_to_get]
obs_type_label = ['CTD','XBT','Profiling float','Mooring','Glider','Drifting Buoy']



for i_obs_type in [4,5]:
    
    print(obs_type_label[i_obs_type])
    meets_condition = ( (data_container_for_year[i_obs_type]['lat']>=LATITUDE_LIM_SOUTH) & (data_container_for_year[i_obs_type]['lat']<=LATITUDE_LIM_NORTH) 
                     & (data_container_for_year[i_obs_type]['lon']>=LONGITUDE_LIM_WEST) & (data_container_for_year[i_obs_type]['lon']<=LONGITUDE_LIM_EAST)  ) 


    casts_in_box = data_container_for_year[i_obs_type]['casts'][meets_condition]

    if meets_condition.sum() != 0:
        temperature_casts_to_get           = data_container_for_year[i_obs_type]['Temperature_cast_index'].isin(casts_in_box)
        temperature_for_casts              = data_container_for_year[i_obs_type]['Temperature'][temperature_casts_to_get]
        temperature_cast_index_for_cast    = data_container_for_year[i_obs_type]['Temperature_cast_index'][temperature_casts_to_get]

        latitude_for_casts   = data_container_for_year[i_obs_type]['lat'].sel(casts=temperature_cast_index_for_cast)
        longitude_for_casts  = data_container_for_year[i_obs_type]['lon'].sel(casts=temperature_cast_index_for_cast)
        time_for_casts       = data_container_for_year[i_obs_type]['time'].sel(casts=temperature_cast_index_for_cast)



        z_casts_to_get         = data_container_for_year[i_obs_type]['z_cast_index'].isin(temperature_cast_index_for_cast)
        z_for_casts            = data_container_for_year[i_obs_type]['z'][z_casts_to_get]
        z_cast_index_for_cast  = data_container_for_year[i_obs_type]['z_cast_index'][z_casts_to_get]

        variable_output_dataarray = xarray.DataArray(latitude_for_casts,dims=['Temperature_obs'],coords={'Temperature_obs':latitude_for_casts['Temperature_obs']}).to_dataset(name='lat')
        variable_output_dataarray['lon'] = xarray.DataArray(longitude_for_casts,dims=['Temperature_obs'],coords={'Temperature_obs':longitude_for_casts['Temperature_obs']})
        variable_output_dataarray['time'] = xarray.DataArray(time_for_casts,dims=['Temperature_obs'],coords={'Temperature_obs':time_for_casts['Temperature_obs']})
        variable_output_dataarray['Temperature'] = xarray.DataArray(temperature_for_casts,dims=['Temperature_obs'],coords={'Temperature_obs':time_for_casts['Temperature_obs']})
        variable_output_dataarray['z'] = xarray.DataArray(z_for_casts,dims=['Temperature_obs'],coords={'Temperature_obs':time_for_casts['Temperature_obs']})
    
    
        variable_output_dataarray.to_netcdf('/tube1/cha674/CARS_2022_ancillary_data/WOD_intermediate/WOD_' +str(year_to_get) + '_Temperature' + obs_type_label[i_obs_type] + '.nc')

Glider
Drifting Buoy


In [10]:
meets_condition.sum()

<xarray.DataArray ()>
array(0)

In [8]:
casts_in_box

ValueError: zero-size array to reduction operation maximum which has no identity

<xarray.DataArray 'casts' (casts: 0)>
array([], dtype=int64)
Coordinates:
    lat      (casts) float32 
    lon      (casts) float32 
    time     (casts) datetime64[ns] ...
Dimensions without coordinates: casts

In [9]:
print(z_for_casts.shape)
print(temperature_for_casts.shape)

(430933,)
(430933,)


In [13]:
latitude_for_casts['Temperature_obs'].values

array([     0,      1,      2, ..., 430930, 430931, 430932])

In [38]:
max_depth = 5

idx = np.nonzero(z_for_casts.values<max_depth)[0]
z_near_surface_2 = z_for_casts[idx]
#z_near_surface = z_for_casts.where(z_for_casts<max_depth,drop=True)
T_near_surface = temperature_for_casts[idx]

#temperature_cast_index_for_cast    = data_container_for_year[i_obs_type]['Temperature_cast_index'][temperature_casts_to_get]



In [39]:
T_near_surface

<xarray.DataArray 'Temperature' (Temperature_obs: 1703)>
array([10.637, 13.293, 13.288, ..., 13.449, 13.449, 13.449], dtype=float32)
Dimensions without coordinates: Temperature_obs
Attributes:
    long_name:            sea_water_temperature
    standard_name:        sea_water_temperature
    units:                degree_C
    grid_mapping:         crs
    ancillary_variables:  Temperature_sigfigs Temperature_WODflag Temperature...

In [ ]:
z_near_surface_2
z_near_surface